In [1]:
import pandas as pd
import glob
import os
import re
from constants import columns_to_keep

### CHANGE THIS LINE WITH THE CORRECT FOLDER
folder_path = "2025"

csv_files = [file for file in glob.glob(os.path.join(folder_path, "**", "*.csv"), recursive=True)
             if "playerpositioning" not in os.path.basename(file).lower()]

df_list = [pd.read_csv(file) for file in csv_files]
final_df = pd.concat(df_list, ignore_index=True)

def impute_missing_columns(df):
    default_values = {
        'PAofInning': 1,
        'BatterTeam': 'GEO_BUL',
        'Inning': 1,
        'Top/Bottom': 'Top',
        'Outs': 1,
        'Balls': 1,
        'Strikes': 1,
        'AutoPitchType': 'NA',
        'PitchCall': 'NA',
        'PlayResult': 'NA',
        'OutsOnPlay': 'NA',
        'RunsScored': 'NA',
        'HomeTeam': 'GEO_PRA',
        'AwayTeam': 'GEO_PRA',
        'Level': 'D1',
        'League': 'SEC',
        'GameUID': 'NA'
    }

    for col, value in default_values.items():
        if col not in df.columns:
            df[col] = value

    if all(col in df.columns for col in ['PitchNo', 'Date', 'Time']):
        df['PitchUID'] = df['PitchNo'].astype(str) + df['Date'].astype(str) + df['Time'].astype(str)
        df['PitchUID'] = df['PitchUID'].apply(lambda x: ''.join(re.findall(r'\d+', x)))

    if 'Date' in df.columns:
        df['UTCDateTime'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%dT00:00:00.000Z')

    return df

final_df = impute_missing_columns(final_df)

pitches_df = pd.read_csv('all_pitches.csv')
final_df = final_df[columns_to_keep]
pitches_df = pd.concat([pitches_df, final_df], ignore_index=True)

pitches_df = pitches_df.drop_duplicates().reset_index(drop=True)
pitches_df.to_csv('all_pitches.csv')

/var/folders/l7/wzwhjfv129l1wmlklwpt_j_r0000gn/T/ipykernel_47803/625577362.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(df_list, ignore_index=True)
/var/folders/l7/wzwhjfv129l1wmlklwpt_j_r0000gn/T/ipykernel_47803/625577362.py:51: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  pitches_df = pd.read_csv('all_pitches.csv')
